In [1]:
import numpy as np
import torch
import torch.utils.data as Data
import torch.nn as nn
from torch.nn import init
import torch.optim as optim

### 生成数据集

In [4]:
num_inputs = 2
num_examples = 1000
true_w = [3,-4]
true_b = -3.5
features = torch.tensor(np.random.normal(0,1,(num_examples,num_inputs)),dtype=torch.float)
labels = features[:,0] * true_w[0] + features[:,1] * true_w[1] + true_b
labels += torch.tensor(np.random.normal(0,0.01,size=labels.size()),dtype=torch.float)

### 读取数据

In [6]:
# import torch.utils.data as Data
batch_size = 10
# 将训练数据的特征和标签组合
dataset = Data.TensorDataset(features,labels)
# 随机读取小批量
data_iter = Data.DataLoader(dataset,batch_size,shuffle=True)

In [7]:
for X,y in data_iter:
    print(X,y)
    break

tensor([[ 0.2647,  0.3498],
        [ 1.0269,  0.0742],
        [-0.9327,  0.1409],
        [ 0.0229, -0.7897],
        [-0.4243, -0.5448],
        [-0.8788,  0.6667],
        [ 0.8371, -1.1136],
        [-0.3440, -0.4907],
        [-1.2125, -0.0908],
        [-1.8074, -0.4315]]) tensor([-4.0934, -0.7140, -6.8637, -0.2868, -2.6207, -8.8040,  3.4659, -2.5682,
        -6.7690, -7.2014])


### 定义模型

首先，导入`torch.nn`模块。实际上，“nn”是neural networks（神经网络）的缩写。顾名思义，该模块定义了大量神经网络的层。之前我们已经用过了`autograd`，而nn就是利用`autograd`来定义模型。

nn的核心数据结构是`Module`，它是一个抽象概念，既可以表示神经网络中的某个层（layer），也可以表示一个包含很多层的神经网络。

在实际使用中，最常见的做法是继承`nn.Module`，撰写自己的网络/层。一个`nn.Module`实例应该包含一些层以及返回输出的前向传播（forward）方法。

下面先来看看如何用`nn.Module`实现一个线性回归模型。

In [8]:
class LinearNet(nn.Module):
    def __init__(self,n_feature):
        super(LinearNet,self).__init__()
        # n_feature个特征
        self.linear = nn.Linear(n_feature,1)
    # forward 定义前向传播
    def forward(self,x):
        y = self.linear(x)
        return y
net = LinearNet(num_inputs)
print(net)# 使用print可以打印出网络的结构

LinearNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


事实上我们还可以用`nn.Sequential`来更加方便地搭建网络，`Sequential`是一个有序的容器，网络层将按照在传入`Sequential`的顺序依次被添加到计算图中。

In [14]:
# 写法一
net = nn.Sequential(
    nn.Linear(num_inputs,1)
    # 此处还可以传入其他层
)

In [15]:
# 写法二
net = nn.Sequential()
net.add_module('linear', nn.Linear(num_inputs, 1))
# net.add_module ......

In [16]:
# 写法三
from collections import OrderedDict
net = nn.Sequential(OrderedDict([
          ('linear', nn.Linear(num_inputs, 1))
          # ......
        ]))

print(net)
print(net[0])

Sequential(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)
Linear(in_features=2, out_features=1, bias=True)


可以通过net.parameters()来查看模型所有的可学习参数，此函数将返回一个生成器。

In [9]:
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[ 0.3122, -0.2966]], requires_grad=True)
Parameter containing:
tensor([0.0357], requires_grad=True)


### 初始化模型参数

In [11]:
# from torch.nn import init
'''
init.normal_(net[0].weight, mean=0, std=0.01)
init.constant_(net[0].bias, val=0)  # 也可以直接修改bias的data: net[0].bias.data.fill_(0)
'''

'\ninit.normal_(net[0].weight, mean=0, std=0.01)\ninit.constant_(net[0].bias, val=0)  # 也可以直接修改bias的data: net[0].bias.data.fill_(0)\n'

> 注：如果这里的`net`是用3.3.3节一开始的代码自定义的，那么上面代码会报错，`net[0].weight`应改为`net.linear.weight`，bias亦然。因为`net[0]`这样根据下标访问子模块的写法只有当`net`是个`ModuleList`或者`Sequential`实例时才可以，详见4.1节。

In [12]:
init.normal_(net.linear.weight,mean=0,std=0.01)
init.constant_(net.linear.bias, val=0)

Parameter containing:
tensor([0.], requires_grad=True)

### 定义损失函数

PyTorch在nn模块中提供了各种损失函数，这些损失函数可看作是一种特殊的层，PyTorch也将这些损失函数实现为nn.Module的子类。我们现在使用它提供的均方误差损失作为模型的损失函数。

In [13]:
loss = nn.MSELoss()

### 定义优化算法

同样，我们也无须自己实现小批量随机梯度下降算法。`torch.optim`模块提供了很多常用的优化算法比如SGD、Adam和RMSProp等。下面我们创建一个用于优化`net`所有参数的优化器实例，并指定学习率为0.03的小批量随机梯度下降（SGD）为优化算法。

In [14]:
# import torch.optim as optim
optimizer = optim.SGD(net.parameters(),lr=0.03)
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    lr: 0.03
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)


我们还可以为不同子网络设置不同的学习率，这在finetune时经常用到。例：

In [15]:
optimizer =optim.SGD([
                # 如果对某个参数不指定学习率，就使用最外层的默认学习率
                {'params': net.subnet1.parameters()}, # lr=0.03
                {'params': net.subnet2.parameters(), 'lr': 0.01}
            ], lr=0.03)

AttributeError: 'LinearNet' object has no attribute 'subnet1'

### 训练模型